In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



In [2]:
# Study data files
crop_production_path = "crop_production_clean.csv"
temp_file_path = "temperature_clean.csv"

In [3]:
crop_pro_results = pd.read_csv(crop_production_path,encoding="ISO-8859-1")
temp_read_results = pd.read_csv(temp_file_path,encoding="ISO-8859-1")
crop_pro_results =crop_pro_results[ ["Area","Item"] + list(crop_pro_results.loc[:,"Y1961":"Y2013"]) ]
crop_pro_results

#rename the columns for crop file 
renamed_crop_column=[]
for col in crop_pro_results.columns:
    renamed_crop_column.append(f'{col}_crop')
    
crop_pro_results.columns =renamed_crop_column


#compare with temp file country names
area_col=crop_pro_results["Area_crop"].unique()



In [4]:
# Temperature data set
temp_avg =temp_read_results[["AverageTemperature","year","Country"]]
temp_avg_df=pd.DataFrame(temp_avg).set_index("year")
temp_avg_df = temp_avg_df.reset_index()#.groupby("Country","year").AverageTemperature.mean()
temp_avg_grouped =pd.DataFrame(temp_avg_df).groupby(["Country","year"]).AverageTemperature.mean()
temp_avg_grouped=temp_avg_grouped.unstack()

#reset index to make country a column for merging
temp_avg_grouped=temp_avg_grouped.reset_index()


# Rename the columns for temp file
renamed_column =[]
for column in temp_avg_grouped.columns:
    renamed_column.append(f'{column}Temp')
    
temp_avg_grouped.columns =renamed_column
temp_avg_sorted=temp_avg_grouped.sort_values("CountryTemp")


#crop_results_sorted =crop_pro_results.sort_values("Area_crop")

#Rename country to match our country of focus
crop_country_list =temp_avg_sorted["CountryTemp"].values
for index,value in enumerate(crop_country_list):
    if value =="United States":#"United States of America"
        print(f'index:{index},value:{value}')
        #rename the country to match crop file for merging
        temp_avg_sorted["CountryTemp"][232]="United States of America"


#compare with crop file columns
#country_col =temp_avg_sorted["CountryTemp"].unique()



index:232,value:United States


C:\Users\momad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
#Merge the two data sets
combined_crop_temp_df= crop_pro_results.merge(temp_avg_sorted, how ="outer",left_on='Area_crop', right_on="CountryTemp", suffixes=(False, False))
combined_crop_temp_df =combined_crop_temp_df.dropna()


In [6]:
# Crop production in the US
usa_prod_df = combined_crop_temp_df.loc[combined_crop_temp_df['Area_crop']=="United States of America"]
usa_prod_df

,Area_crop,Item_crop,Y1961_crop,Y1962_crop,Y1963_crop,Y1964_crop,Y1965_crop,Y1966_crop,Y1967_crop,Y1968_crop,...,2002Temp,2003Temp,2004Temp,2005Temp,2006Temp,2007Temp,2008Temp,2009Temp,2010Temp,2011Temp
2929,United States of America,Wheat and products,13439.0,13314.0,13320.0,13525.0,13510.0,13479.0,13717.0,13840.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
2930,United States of America,Rice (Milled Equivalent),481.0,574.0,518.0,564.0,620.0,589.0,611.0,645.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
2931,United States of America,Barley and products,131.0,131.0,152.0,152.0,152.0,152.0,180.0,174.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
2932,United States of America,Maize and products,1470.0,1499.0,1526.0,1570.0,1628.0,1714.0,1694.0,1489.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
2933,United States of America,Rye and products,112.0,119.0,119.0,122.0,116.0,122.0,125.0,135.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,United States of America,Offals,376.0,381.0,390.0,391.0,397.0,405.0,425.0,439.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
3001,United States of America,Animal fats,2106.0,2219.0,2132.0,2077.0,2060.0,1848.0,1935.0,1995.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
3002,United States of America,Eggs,3342.0,3409.0,3424.0,3521.0,3526.0,3559.0,3695.0,3667.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917
3003,United States of America,Milk - Excluding Butter,50289.0,49326.0,50260.0,50967.0,50757.0,51203.0,50105.0,50615.0,...,9.816167,9.755083,9.6675,9.892667,9.9745,9.8375,8.951417,9.137083,9.505583,9.549917


In [7]:
Tot_usa_prod_df =usa_prod_df[list(usa_prod_df.loc[:,"Area_crop":"Y2011_crop"])]
Tot_usa_prod_df["Results"]= Tot_usa_prod_df.apply(lambda column: column['Y1961_crop':'Y2011_crop'].sum(),axis = 1)
Tot_usa_prod_df.head()

C:\Users\momad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Area_crop,Item_crop,Y1961_crop,Y1962_crop,Y1963_crop,Y1964_crop,Y1965_crop,Y1966_crop,Y1967_crop,Y1968_crop,...,Y2003_crop,Y2004_crop,Y2005_crop,Y2006_crop,Y2007_crop,Y2008_crop,Y2009_crop,Y2010_crop,Y2011_crop,Results
2929,United States of America,Wheat and products,13439.0,13314.0,13320.0,13525.0,13510.0,13479.0,13717.0,13840.0,...,24801.0,24511.0,24869.0,25250.0,25793.0,25302.0,24959.0,25167.0,25050.0,980782.0
2930,United States of America,Rice (Milled Equivalent),481.0,574.0,518.0,564.0,620.0,589.0,611.0,645.0,...,1827.0,1943.0,2021.0,2075.0,2104.0,2104.0,2118.0,2194.0,2127.0,68559.0
2931,United States of America,Barley and products,131.0,131.0,152.0,152.0,152.0,152.0,180.0,174.0,...,153.0,158.0,152.0,162.0,164.0,169.0,166.0,167.0,164.0,7905.0
2932,United States of America,Maize and products,1470.0,1499.0,1526.0,1570.0,1628.0,1714.0,1694.0,1489.0,...,3879.0,3879.0,3887.0,3887.0,3887.0,3887.0,3913.0,3954.0,3969.0,137191.0
2933,United States of America,Rye and products,112.0,119.0,119.0,122.0,116.0,122.0,125.0,135.0,...,83.0,83.0,84.0,84.0,85.0,84.0,83.0,81.0,85.0,4255.0


In [ ]:
sorted_Tot_usa_prod_df = Tot_usa_prod_df.sort_values(by=['Results'], ascending=False)
#maize_new = final_df[['Area', 'maize production']].copy()
clean_df=sorted_Tot_usa_prod_df.head(10)

In [ ]:
clean_df.drop_duplicates()

In [ ]:
clean_df.groupby("Area_crop").sum()